# 여자배구 데이터 분석, 경기 결과 예측 프로젝트

## 라이브러리 import

In [1]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import findspark as fs

fs.init()

In [2]:
# spark 생성

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## 데이터 전처리 pt.2 - 22/03/17

In [21]:
# 파일 불러오기
data = spark.read.csv("hdfs://localhost:9000/data/player_rate.csv", header="true", inferSchema="true")
data.show()

+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|  name|        team|pos| score|error|setcount|at_srate|at_rate| sv| set| bl|rs_rate|  dg|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|임명옥|한국도로공사| Li|1475.0|    3|     117|     0.0|    0.0|0.0|1.05|0.0|  55.24| 5.9|
|신연경| IBK기업은행| Li|1158.6|    3|     112|     0.0|    0.0|0.0| 0.6|0.0|  39.43|5.28|
|김연견|    현대건설| Li|1099.0|    4|     113|     0.0|    0.0|0.0|0.79|0.0|   39.2|4.96|
|오지영|    GS칼텍스| Li| 880.6|    2|      94|     0.0|    0.0|0.0|0.51|0.0|  39.37|3.99|
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|    0.0|0.0| 0.7|0.0|  32.04| 3.1|
|  노란| KGC인삼공사| Li| 819.8|    5|      83|     0.0|    0.0|0.0|0.65|0.0|  40.62|5.01|
|김해란|    흥국생명| Li| 596.4|    2|      58|     0.0|    0.0|0.0|1.02|0.0|  31.28|5.78|
|도수빈|    흥국생명| Li| 589.2|    3|      60|     0.0|    0.0|0.0|0.63|0.0|  38.38| 4.9|
|김세인|페퍼저축은행| Li| 334.4|    0|      84|     0.0|    0.0|0.

In [22]:
# 불러온 데이터 테이블로 만들기
data.createOrReplaceTempView("player")

In [23]:
player = spark.sql("""
          select name,team,pos,at_srate,at_rate,sv,set,bl,rs_rate,dg
          from player
          """)

player.createOrReplaceTempView("player")

player.show()

+------+------------+---+--------+-------+---+----+---+-------+----+
|  name|        team|pos|at_srate|at_rate| sv| set| bl|rs_rate|  dg|
+------+------------+---+--------+-------+---+----+---+-------+----+
|임명옥|한국도로공사| Li|     0.0|    0.0|0.0|1.05|0.0|  55.24| 5.9|
|신연경| IBK기업은행| Li|     0.0|    0.0|0.0| 0.6|0.0|  39.43|5.28|
|김연견|    현대건설| Li|     0.0|    0.0|0.0|0.79|0.0|   39.2|4.96|
|오지영|    GS칼텍스| Li|     0.0|    0.0|0.0|0.51|0.0|  39.37|3.99|
|문슬기|페퍼저축은행| Li|     0.0|    0.0|0.0| 0.7|0.0|  32.04| 3.1|
|  노란| KGC인삼공사| Li|     0.0|    0.0|0.0|0.65|0.0|  40.62|5.01|
|김해란|    흥국생명| Li|     0.0|    0.0|0.0|1.02|0.0|  31.28|5.78|
|도수빈|    흥국생명| Li|     0.0|    0.0|0.0|0.63|0.0|  38.38| 4.9|
|김세인|페퍼저축은행| Li|     0.0|    0.0|0.0|0.23|0.0|  30.56| 2.2|
|채선아| KGC인삼공사| Li|     0.0|    0.0|0.0|0.68|0.0|  39.34|3.78|
|한수진|    GS칼텍스| Li|     0.0|    0.0|0.0|0.25|0.0|    0.0|2.25|
|한다혜|    GS칼텍스| Li|     0.0|    0.0|0.0|0.27|0.0|  51.79| 1.6|
|김수빈| IBK기업은행| Li|     0.0|    0.0|0.0|0.08|0.0|  2

In [20]:
player = spark.sql("""
          SELECT * FROM player WHERE name!='하혜진'
          """)

player.createOrReplaceTempView("player")

spark.sql("SELECT * FROM player WHERE pos='R'").show()

+--------+------------+---+--------+-------+----+----+----+-------+----+
|    name|        team|pos|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+--------+------------+---+--------+-------+----+----+----+-------+----+
|    캣벨|    흥국생명|  R|    36.6|  29.45|0.16|0.08|0.42|    0.0|3.16|
|    켈시|한국도로공사|  R|   42.19|  36.94|0.12|0.01| 0.6|    0.0|2.51|
|    모마|    GS칼텍스|  R|   47.58|  41.27| 0.4|0.11|0.36|  33.33|2.38|
|  옐레나| KGC인삼공사|  R|   39.67|  34.75|0.16| 0.1|0.59|  14.29|3.19|
|  야스민|    현대건설|  R|   42.86|  35.34|0.44|0.09|0.43|  100.0|2.05|
|엘리자벳|페퍼저축은행|  R|   41.45|  32.89|0.17|0.03|0.49| -33.33|2.01|
|  문정원|한국도로공사|  R|   29.69|  24.89|0.14| 0.3|0.06|  38.73|2.38|
|    라셈| IBK기업은행|  R|   34.82|  27.47|0.17|0.06|0.23|    0.0|1.87|
|  김다은|    흥국생명|  R|   33.02|  24.19|0.14|0.22| 0.3|  18.69|2.19|
|  황연주|    현대건설|  R|   38.82|  35.29|0.07|0.04|0.11|    0.0|0.79|
|  이은지|페퍼저축은행|  R|     0.0|    0.0|0.01| 0.0| 0.0|   50.0| 0.2|
|  문지윤|    GS칼텍스|  R|    48.0|   40.0| 0.0| 0.0|0.13|    0.

In [25]:
ha = spark.sql("""
          SELECT name,team,'C' as pos,at_srate,at_rate,sv,set,bl,rs_rate,dg
          FROM player
          WHERE name='하혜진'
          UNION
          SELECT * FROM player WHERE name!='하혜진'
          """)
ha.show()

ha.createOrReplaceTempView("ha")

+------+------------+---+--------+-------+----+----+----+-------+----+
|  name|        team|pos|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+------------+---+--------+-------+----+----+----+-------+----+
|한송이| KGC인삼공사|  C|   39.89|  35.11|0.08|0.23|0.53|  21.05|1.31|
|이예담|한국도로공사|  C|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|이나연|    현대건설|  S|     0.0|    0.0|0.02|3.81| 0.0|    0.0|0.81|
|차유정|    GS칼텍스| Li|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|  이진| IBK기업은행|  S|     0.0| -14.29| 0.0|3.09|0.06|    0.0|0.69|
|양효진|    현대건설|  C|   52.48|  47.92|0.21|0.24|0.74|  40.54|1.44|
|조송화| IBK기업은행|  S|   15.38|   7.69|0.09|6.73|0.09|    0.0|2.68|
|이현지|    현대건설|  L|     0.0|    0.0|0.17| 0.0| 0.0|   25.0|0.08|
|한송희|한국도로공사|  L|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|  이현|페퍼저축은행|  S|   21.62|  16.22|0.13|8.37|0.04|    0.0|2.11|
|하혜진|페퍼저축은행|  C|   33.89|  27.52|0.15|0.09|0.39|   10.0|1.14|
|육서영| IBK기업은행|  L|    30.0|  18.24|0.03|0.05| 0.0|  16.36|1.18|
|채선아| KGC인삼공사|  L|     0

In [32]:
spark.sql("SELECT * FROM ha WHERE pos='C'").show(5)

+------+------------+---+--------+-------+----+----+----+-------+----+
|  name|        team|pos|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+------------+---+--------+-------+----+----+----+-------+----+
|한송이| KGC인삼공사|  C|   39.89|  35.11|0.08|0.23|0.53|  21.05|1.31|
|이예담|한국도로공사|  C|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|양효진|    현대건설|  C|   52.48|  47.92|0.21|0.24|0.74|  40.54|1.44|
|하혜진|페퍼저축은행|  C|   33.89|  27.52|0.15|0.09|0.39|   10.0|1.14|
|정시영|    현대건설|  C|    50.0|    0.0| 0.0|0.11| 0.0|    0.0|0.22|
+------+------------+---+--------+-------+----+----+----+-------+----+
only showing top 5 rows



In [31]:
ha = spark.sql("""
          SELECT * FROM ha
          ORDER BY pos
          """)
ha.show()

ha.createOrReplaceTempView("player")

+------+------------+---+--------+-------+----+----+----+-------+----+
|  name|        team|pos|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+------------+---+--------+-------+----+----+----+-------+----+
|오세연|    GS칼텍스|  C|   100.0|  100.0| 0.0| 0.0| 0.0|    0.0| 0.0|
|김현정| IBK기업은행|  C|   28.57|  17.14|0.04|0.07|0.11|  -25.0|0.52|
|이다현|    현대건설|  C|   46.88|  40.63|0.09|0.15|0.74|  37.04|1.05|
|이주아|    흥국생명|  C|   36.29|   27.7|0.22|0.18|0.72|  30.56| 1.0|
|박은진| KGC인삼공사|  C|   42.24|  38.36|0.11|0.16|0.44|  13.33|0.81|
|정호영| KGC인삼공사|  C|   48.37|  42.93|0.11|0.08|0.53|  11.11|0.88|
|양효진|    현대건설|  C|   52.48|  47.92|0.21|0.24|0.74|  40.54|1.44|
|최정민| IBK기업은행|  C|    34.4|  27.98|0.13|0.08|0.29|    0.0|0.89|
|하유정|한국도로공사|  C|   45.45|  36.36| 0.0| 0.0| 0.0|    0.0| 0.4|
|김유리|    GS칼텍스|  C|   40.82|  37.76|0.11|0.06|0.15|    0.0|0.58|
|김채연|    흥국생명|  C|   34.24|  27.24| 0.1|0.13| 0.4|   35.0|1.15|
|하혜진|페퍼저축은행|  C|   33.89|  27.52|0.15|0.09|0.39|   10.0|1.14|
|이예담|한국도로공사|  C|     

In [33]:
ha.count()

141

In [43]:
player = ha

## 데이터 시각화 - 22/3/17

In [40]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [41]:
pip install seaborn

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-models-official 2.3.0 requires Cython, which is not installed.
resampy 0.2.2 requires numba>=0.32, which is not installed.
librosa 0.8.1 requires numba>=0.43.0, which is not installed.
librosa 0.8.1 requires scikit-learn!=0.19.0,>=0.14.0, which is not installed.


In [42]:
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
'''at_rate_show = player['pos'].plot( kind='hist', facecolor='lightblue')
at_rate_show.set_title("Player's Position")
at_rate_show.set_xlabel('Player Position')
at_rate_show.set_ylabel('Counts')
plt.suptitle('')'''
plt.show()

In [44]:
# 선수 데이터 저장 @ 하둡

player_rate2.write.format('csv').save('hdfs://localhost:9000/data/player_rate.csv',header='true')

In [45]:
data = spark.read.csv("hdfs://localhost:9000/data/player_rate.csv", header='true', inferSchema='true')
data.show(5)

+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|  name|        team|pos| score|error|setcount|at_srate|at_rate| sv| set| bl|rs_rate|  dg|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
|임명옥|한국도로공사| Li|1475.0|    3|     117|     0.0|    0.0|0.0|1.05|0.0|  55.24| 5.9|
|신연경| IBK기업은행| Li|1158.6|    3|     112|     0.0|    0.0|0.0| 0.6|0.0|  39.43|5.28|
|김연견|    현대건설| Li|1099.0|    4|     113|     0.0|    0.0|0.0|0.79|0.0|   39.2|4.96|
|오지영|    GS칼텍스| Li| 880.6|    2|      94|     0.0|    0.0|0.0|0.51|0.0|  39.37|3.99|
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|    0.0|0.0| 0.7|0.0|  32.04| 3.1|
+------+------------+---+------+-----+--------+--------+-------+---+----+---+-------+----+
only showing top 5 rows



In [46]:
# 구단별 데이터 저장 @ 하둡

hgun_rate.write.format('csv').save('hdfs://localhost:9000/data/hgun_rate.csv',header='true')
dogong_rate.write.format('csv').save('hdfs://localhost:9000/data/dogong_rate.csv',header='true')
gscal_rate.write.format('csv').save('hdfs://localhost:9000/data/gscal_rate.csv',header='true')
insam_rate.write.format('csv').save('hdfs://localhost:9000/data/insam_rate.csv',header='true')
ibk_rate.write.format('csv').save('hdfs://localhost:9000/data/ibk_rate.csv',header='true')
hguk_rate.write.format('csv').save('hdfs://localhost:9000/data/hguk_rate.csv',header='true')
pepper_rate.write.format('csv').save('hdfs://localhost:9000/data/pepper_rate.csv',header='true')

In [47]:
# 데이터 잘 저장됐는지 확인

data = spark.read.csv("hdfs://localhost:9000/data/pepper_rate.csv", header='true', inferSchema="true")
data.show(5)

+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|  name|        team|pos| score|error|setcount|at_srate|at_rate|  sv| set|  bl|rs_rate|  dg|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
|문슬기|페퍼저축은행| Li| 841.8|    2|      99|     0.0|    0.0| 0.0| 0.7| 0.0|  32.04| 3.1|
|김세인|페퍼저축은행| Li| 334.4|    0|      84|     0.0|    0.0| 0.0|0.23| 0.0|  30.56| 2.2|
|이은지|페퍼저축은행| Li|   7.8|    0|       4|     0.0|    0.0| 0.0| 0.0| 0.0|    0.0| 0.5|
|이한비|페퍼저축은행|  L|1930.4|   91|     105|   30.09|  25.03| 0.1| 0.2|0.12|  29.35|3.18|
|박경현|페퍼저축은행|  L|1714.8|   94|      96|   35.57|  30.66|0.16|0.23| 0.1|  22.13|2.86|
+------+------------+---+------+-----+--------+--------+-------+----+----+----+-------+----+
only showing top 5 rows

